# Análisis de las imágenes de los troncos

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import PIL.ExifTags
import pandas as pd

from math import cos, sin
from collections import defaultdict

In [ ]:
image = cv2.imread('../Data/RGB/30_0.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(image)

# Datos EXIF de las imágenes (por si hay algo relevante)

In [ ]:
def get_exif(image_path):
    img = PIL.Image.open(image_path)
    exif = {
        PIL.ExifTags.TAGS[k]: v
        for k, v in img._getexif().items()
        if k in PIL.ExifTags.TAGS
    }
    
    return exif

In [ ]:
get_exif('../Data/RGB/30_0.jpg')

# Static saliency

In [ ]:
saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
(success, saliencyMap) = saliency.computeSaliency(image)
saliencyMap = (saliencyMap * 255).astype("uint8")plt.imshow(image)
plt.imshow(saliencyMap)

# Fine-grained saliency

In [ ]:
saliency = cv2.saliency.StaticSaliencyFineGrained_create()
(success, saliencyMap) = saliency.computeSaliency(image)

threshMap = cv2.threshold(saliencyMap.astype("uint8"), 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
plt.imshow(saliencyMap)

# Contours (bbox)

In [ ]:
def bbox_area(bbox):
    return bbox[1][0] * bbox[1][1]

def subimage(image, center, theta, width, height):
    theta *= 3.14159 / 180 # convert to rad

    v_x = (cos(theta), sin(theta))
    v_y = (-sin(theta), cos(theta))
    s_x = center[0] - v_x[0] * ((width-1) / 2) - v_y[0] * ((height-1) / 2)
    s_y = center[1] - v_x[1] * ((width-1) / 2) - v_y[1] * ((height-1) / 2)

    mapping = np.array([[v_x[0],v_y[0], s_x],
                        [v_x[1],v_y[1], s_y]])

    return cv2.warpAffine(image,mapping,(width, height),flags=cv2.WARP_INVERSE_MAP,borderMode=cv2.BORDER_REPLICATE)

def bbox_contours(image_path, draw=True):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 100, 255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = [cont for cont in contours if len(cont) >= 10000]
    contours.sort(key = len)
    contours = contours[:-1]
    
    boundRects = []
    for i, c in enumerate(contours):
        boundRects.append(cv2.minAreaRect(c))
        
    boundRectsAreas = [bbox_area(rect) for rect in boundRects]
    maxBoundRect = np.argmax(boundRectsAreas)
    
    if draw:
        contours = [contours[maxBoundRect]]
        boundRects = [boundRects[maxBoundRect]]
        
        imcopy = image.copy()
        cv2.drawContours(imcopy, contours, contourIdx=-1, color=(0, 255, 0), thickness=10, lineType=cv2.LINE_AA)
        for i in range(len(contours)):
            box = cv2.boxPoints(boundRects[i])
            box = np.intp(box) #np.intp: Integer used for indexing (same as C ssize_t; normally either int32 or int64)
            cv2.drawContours(imcopy, [box], 0, (255,0,0), thickness=50)

        plt.figure(figsize = (9, 12))
        plt.imshow(imcopy)
        plt.savefig('../Figures/bbox_contour.jpg')
    
    else:
        bestbBox = boundRects[maxBoundRect]
        img_crop = subimage(image, bestbBox[0], bestbBox[2], int(bestbBox[1][0]), int(bestbBox[1][1]))
        return img_crop


rotations = ['0', '90', '180', '270']
ids = ['Ny', '5', '6', '30', '41']

for rot in rotations:
    for pid in ids:
        path = f'RGB/{pid}_{rot}.jpg'
        pole_crop = bbox_contours(path, draw=False)
        pole_crop = cv2.cvtColor(pole_crop, cv2.COLOR_BGR2RGB)
        if (pole_crop.shape[0] < pole_crop.shape[1]):
            pole_crop = np.flip(np.swapaxes(pole_crop, 0, 1), 1)
        cv2.imwrite(f'../Data/RGB/crops/{pid}_{rot}_crop.jpg', pole_crop)

# Contours (masked)

In [ ]:
def keep_largest_mask_component(mask):
    mask = mask.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=4)
    sizes = stats[:, -1]

    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255
    return img2

def mask_contours(image_path, draw=True):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 100, 255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = [cont for cont in contours if len(cont) >= 10000]
    contours.sort(key = len)
    contours = contours[:-1]
    
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, 255, -1)
    mask = keep_largest_mask_component(mask[:,:,0])
    mask = np.repeat(mask[...,np.newaxis], 3, axis=2)
    out = np.zeros_like(image)
    out[mask == 255] = image[mask == 255]

    if draw:
        plt.figure(figsize = (9, 12))
        plt.imshow(out)
    
    else:
        return out, mask


rotations = ['0', '90', '180', '270']
ids = ['Ny', '5', '6', '30', '41']

for rot in rotations:
    for pid in ids:
        path = f'../Data/RGB/{pid}_{rot}.jpg'
        pole_crop, pole_mask = mask_contours(path, draw=False)
        cv2.imwrite(f'../Data/RGB/crops/{pid}_{rot}_crop_masked.jpg', pole_crop)
        cv2.imwrite(f'../Data/RGB/crops/{pid}_{rot}_crop_mask.jpg', pole_mask)

# Histogramas de color de los troncos

In [ ]:
def extract_histograms(image):
    bgr_planes = cv2.split(image)
    histSize = 256
    histRange = (0, 256) # the upper boundary is exclusive
    accumulate = False
    b_hist = cv2.calcHist(bgr_planes, [0], None, [histSize], histRange, accumulate=accumulate)
    g_hist = cv2.calcHist(bgr_planes, [1], None, [histSize], histRange, accumulate=accumulate)
    r_hist = cv2.calcHist(bgr_planes, [2], None, [histSize], histRange, accumulate=accumulate)
    hist_w = 512
    hist_h = 400
    bin_w = int(round( hist_w/histSize ))
    histImage = np.zeros((hist_h, hist_w, 3), dtype=np.uint8)
    cv2.normalize(b_hist, b_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(g_hist, g_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(r_hist, r_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    
    return r_hist, g_hist, b_hist

def visualize_histograms(color_scheme=cv2.COLOR_BGR2RGB):
    fig, axes = plt.subplots(5, 8, figsize=(20,16))
    pole_ids = ['Ny',30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']

    for pole_idx, pole_id in enumerate(pole_ids):
        for rotation_idx, rotation in enumerate([0, 90, 180, 270]):
            img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop.jpg')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            r_hist, g_hist, b_hist = extract_histograms(img)
            axes[pole_idx, rotation_idx*2].plot(range(256), r_hist, color='red')
            axes[pole_idx, rotation_idx*2].plot(range(256), g_hist, color='green')
            axes[pole_idx, rotation_idx*2].plot(range(256), b_hist, color='blue')
            axes[pole_idx, rotation_idx*2+1].imshow(img)
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])

    fig.suptitle('Color histograms')

    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=50, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]

    plt.savefig('../Figures/histograms_color.jpg')
    
visualize_histograms()

In [ ]:
def extract_histograms_gray(image):
    histSize = 256
    histRange = (0, 256) # the upper boundary is exclusive
    accumulate = False
    hist = cv2.calcHist([image], [0], None, [histSize], histRange, accumulate=accumulate)
    hist_w = 512
    hist_h = 400
    bin_w = int(round( hist_w/histSize ))
    histImage = np.zeros((hist_h, hist_w, 3), dtype=np.uint8)
    cv2.normalize(hist, hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    
    return hist

def visualize_histograms_gray():
    fig, axes = plt.subplots(5, 8, figsize=(20,16))
    pole_ids = ['Ny',30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    pole_stdevs = []

    for pole_idx, pole_id in enumerate(pole_ids):
        pole_stdevs.append([])
        for rotation_idx, rotation in enumerate([0, 90, 180, 270]):
            img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop.jpg')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            hist = extract_histograms_gray(img)
            axes[pole_idx, rotation_idx*2].plot(range(256), hist, color='black')
            axes[pole_idx, rotation_idx*2+1].imshow(img, cmap='gray')
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])
            polrot_mean, polrot_std = np.mean(hist), np.std(hist)
            pole_stdevs[pole_idx].append(polrot_std)

    fig.suptitle('Color histograms')

    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}\nMax deviation: {max(pole_stdevs[i]):.2f}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]

    plt.savefig('../Figures/histograms_gray.jpg')
    
visualize_histograms_gray()

# Color histograms (masked poles)

In [ ]:
def extract_histograms_mask(image, mask):
    bgr_planes = cv2.split(image)
    histSize = 256
    histRange = (0, 256) # the upper boundary is exclusive
    accumulate = False
    b_hist = cv2.calcHist(bgr_planes, [0], mask, [histSize], histRange, accumulate=accumulate)
    g_hist = cv2.calcHist(bgr_planes, [1], mask, [histSize], histRange, accumulate=accumulate)
    r_hist = cv2.calcHist(bgr_planes, [2], mask, [histSize], histRange, accumulate=accumulate)
    hist_w = 512
    hist_h = 400
    bin_w = int(round( hist_w/histSize ))
    histImage = np.zeros((hist_h, hist_w, 3), dtype=np.uint8)
    cv2.normalize(b_hist, b_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(g_hist, g_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(r_hist, r_hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    
    return r_hist, g_hist, b_hist

def visualize_histograms(color_scheme=cv2.COLOR_BGR2RGB):
    fig, axes = plt.subplots(5, 8, figsize=(20,16))
    pole_ids = [0,30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    
    data = defaultdict(list)
    data['id'] = [0,30,41,6,5]

    for pole_idx, pole_id in enumerate(pole_ids):
        for rotation_idx, rotation in enumerate([0, 90, 180, 270]):
            img = cv2.imread(f'../../RGB/sam_crops/{pole_id}_{rotation}_masked.jpg')
            mask = cv2.imread(f'../../RGB/sam_crops/{pole_id}_{rotation}_mask.jpg')[:,:,0]
            r_hist, g_hist, b_hist = extract_histograms_mask(img, mask)
            axes[pole_idx, rotation_idx*2].plot(range(256), r_hist, color='red')
            axes[pole_idx, rotation_idx*2].plot(range(256), g_hist, color='green')
            axes[pole_idx, rotation_idx*2].plot(range(256), b_hist, color='blue')
            axes[pole_idx, rotation_idx*2+1].imshow(img)
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])
            extent = axes[pole_idx, rotation_idx*2].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
            plt.savefig(f"../Figures/histograms/{pole_id}_{rotation}.svg", bbox_inches=extent)
            
            data[f'r_hist_mean_{rotation}_degrees'].append(np.mean(r_hist))
            data[f'r_hist_std_{rotation}_degrees'].append(np.std(r_hist))
            data[f'g_hist_mean_{rotation}_degrees'].append(np.mean(g_hist))
            data[f'g_hist_std_{rotation}_degrees'].append(np.std(g_hist))
            data[f'b_hist_mean_{rotation}_degrees'].append(np.mean(b_hist))
            data[f'b_hist_std_{rotation}_degrees'].append(np.std(b_hist))

    fig.suptitle('Color histograms')

    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=50, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]

    plt.savefig('../Figures/histograms_color.jpg')
    
    return data

In [ ]:
data = visualize_histograms()
#df_kernel = pd.DataFrame(data)
#df_kernel.to_csv(f'../Features/rgb_histograms.csv')

In [ ]:
def extract_histograms_gray_masked(image, mask):
    histSize = 256
    histRange = (0, 256) # the upper boundary is exclusive
    accumulate = False
    hist = cv2.calcHist([image], [0], mask, [histSize], histRange, accumulate=accumulate)
    hist_w = 512
    hist_h = 400
    bin_w = int(round( hist_w/histSize ))
    histImage = np.zeros((hist_h, hist_w, 3), dtype=np.uint8)
    cv2.normalize(hist, hist, alpha=0, beta=hist_h, norm_type=cv2.NORM_MINMAX)
    
    return hist

def visualize_histograms_gray():
    fig, axes = plt.subplots(5, 8, figsize=(20,16))
    pole_ids = ['Ny',30,41,6,5]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    pole_stdevs = []
    
    data = defaultdict(list)
    data['id'] = [0,30,41,6,5]

    for pole_idx, pole_id in enumerate(pole_ids):
        pole_stdevs.append([])
        for rotation_idx, rotation in enumerate([0, 90, 180, 270]):
            img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop_masked.jpg')
            mask = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop_mask.jpg')[:,:,0]
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            hist = extract_histograms_gray_masked(img, mask)
            axes[pole_idx, rotation_idx*2].plot(range(256), hist, color='black')
            axes[pole_idx, rotation_idx*2+1].imshow(img, cmap='gray')
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])
            polrot_mean, polrot_std = np.mean(hist), np.std(hist)
            pole_stdevs[pole_idx].append(polrot_std)
            
            data[f'gray_hist_mean_{rotation}_degrees'].append(np.mean(hist))
            data[f'gray_hist_std_{rotation}_degrees'].append(np.std(hist))

    fig.suptitle('Color histograms')

    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]

    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}\nMax deviation: {max(pole_stdevs[i]):.2f}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]

    plt.savefig('../Figures/histograms_gray.jpg')
    
    return data

In [ ]:
data = visualize_histograms_gray()
df_gray = pd.DataFrame(data)
df_gray.to_csv(f'../Features/gray_histograms.csv')